## Results of ES & EZ

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as si
import time

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

import tensorflow as tf
from tensorflow import keras

from tabulate import tabulate
sns.set()
np.set_printoptions(precision = 3, suppress = True)

In [2]:
ES_EZ = pd.read_csv('/Users/gaojinglun/Desktop/RSG/ES_EZ.csv')
ES_EZ = ES_EZ.drop(['Unnamed: 0'], axis = 1)
ES_EZ.head()

,Trade.Date,Put.Call,Strike.Price,Contract.Year,Contract.Month,Settlement,Open.Interest,Delta,Implied.Volatility,Last.Trade.Date,futures.price,Time.to.maturity,Risk.Free.Rate
0,20200102,C,2200,2020,3,1059.0,826.0,0.99787,0.338267,20200320,3259.0,0.213699,0.0154
1,20200102,C,2250,2020,3,1009.1,728.0,0.99661,0.326829,20200320,3259.0,0.213699,0.0154
2,20200102,C,2270,2020,3,989.1,1.0,0.99586,0.323456,20200320,3259.0,0.213699,0.0154
3,20200102,C,2300,2020,3,959.2,212.0,0.99474,0.317834,20200320,3259.0,0.213699,0.0154
4,20200102,C,2310,2020,3,949.2,42.0,0.99466,0.314282,20200320,3259.0,0.213699,0.0154


In [3]:
print('There are {} call options and {} put options in the ES_EZ'.format(
    np.sum(ES_EZ['Put.Call'] == 'C'), 
    ES_EZ.shape[0] -np.sum(ES_EZ['Put.Call'] == 'C')))

There are 13708 call options and 16263 put options in the ES_EZ


In [4]:
df = ES_EZ[['Put.Call', 'Strike.Price', 'Settlement', 'Open.Interest', 
            'Delta', 'Implied.Volatility', 'futures.price', 'Time.to.maturity', 'Risk.Free.Rate']]
df.head()

,Put.Call,Strike.Price,Settlement,Open.Interest,Delta,Implied.Volatility,futures.price,Time.to.maturity,Risk.Free.Rate
0,C,2200,1059.0,826.0,0.99787,0.338267,3259.0,0.213699,0.0154
1,C,2250,1009.1,728.0,0.99661,0.326829,3259.0,0.213699,0.0154
2,C,2270,989.1,1.0,0.99586,0.323456,3259.0,0.213699,0.0154
3,C,2300,959.2,212.0,0.99474,0.317834,3259.0,0.213699,0.0154
4,C,2310,949.2,42.0,0.99466,0.314282,3259.0,0.213699,0.0154


In [5]:
ES_EZ_call = df[df['Put.Call'] == 'C']
ES_EZ_put = df[df['Put.Call'] == 'P']

In [6]:
ES_EZ_call.describe()

,Strike.Price,Settlement,Open.Interest,Delta,Implied.Volatility,futures.price,Time.to.maturity,Risk.Free.Rate
count,13708.000000,13708.000000,13705.000000,13708.000000,13708.000000,13708.000000,13708.000000,13708.000000
mean,3123.002261,409.944996,1365.515213,0.572225,0.326937,3285.233528,0.301698,0.002869
std,679.794125,454.275184,2457.997337,0.361411,0.178820,416.592958,0.245081,0.004624
min,100.000000,0.000000,0.000000,0.000000,0.085464,2381.500000,0.008219,0.000200
25%,2670.000000,36.000000,39.000000,0.211120,0.202784,3053.300000,0.095890,0.000900
50%,3140.000000,260.550000,395.000000,0.667915,0.283317,3294.100000,0.257534,0.001100
75%,3600.000000,649.000000,1620.000000,0.912740,0.388412,3623.000000,0.427397,0.001600
max,5000.000000,3226.900000,35941.000000,1.000000,1.781009,3967.600000,0.964384,0.016000


In [7]:
ES_EZ_put.describe()

,Strike.Price,Settlement,Open.Interest,Delta,Implied.Volatility,futures.price,Time.to.maturity,Risk.Free.Rate
count,16263.000000,16263.000000,16263.000000,16263.000000,16263.000000,16263.000000,16263.000000,16263.000000
mean,2686.731845,126.175869,2064.253766,0.271016,0.411271,3291.804101,0.313604,0.002916
std,849.753717,194.278742,3373.751461,0.311253,0.237628,409.744382,0.250395,0.004684
min,100.000000,0.000000,0.000000,0.000000,0.086024,2381.500000,0.008219,0.000200
25%,2175.000000,5.300000,255.000000,0.018800,0.241178,3053.300000,0.095890,0.000900
50%,2790.000000,42.300000,931.000000,0.128570,0.348741,3294.300000,0.260274,0.001100
75%,3300.000000,165.500000,2313.500000,0.449815,0.506372,3623.000000,0.452055,0.001600
max,5000.000000,1603.300000,45609.000000,1.000000,2.168625,3967.600000,0.964384,0.016000


In [8]:
def black_scholes_call_option(S, K, T, q, r, sigma):
    '''
    S: Stock price
    K: Strike price
    T: Maturity
    q: Dividend rate
    r: Risk free rate
    sigma: Volatility
    '''
    
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    call = (S * np.exp(-q * T) * si.norm.cdf(d1) - K * np.exp(-r * T) * si.norm.cdf(d2))
    
    return call

def black_scholes_put_option(S, K, T, q, r, sigma):
    '''
    S: Stock price
    K: Strike price
    T: Maturity
    q: Dividend rate
    r: Risk free rate
    sigma: Volatility
    '''
    
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    put = K * np.exp(-r * T) * si.norm.cdf(-d2) - S * np.exp(-q * T) * si.norm.cdf(-d1)
    
    return put

In [9]:
BS_Call = df[['Strike.Price', 'Settlement', 'Implied.Volatility', 
              'futures.price', 'Time.to.maturity', 'Risk.Free.Rate']] 
BS_Call = BS_Call[df['Put.Call'] == 'C']

BS_Putt = df[['Strike.Price', 'Settlement', 'Implied.Volatility', 
              'futures.price', 'Time.to.maturity', 'Risk.Free.Rate']] 
BS_Putt = BS_Putt[df['Put.Call'] == 'P']

### Results of BS

In [10]:
# Test the performances of BS model
BS_Call_Pred = black_scholes_call_option(
    BS_Call['futures.price'], 
    BS_Call['Strike.Price'], 
    BS_Call['Time.to.maturity'], 
    np.zeros(BS_Call.shape[0]), 
    BS_Call['Risk.Free.Rate'], 
    BS_Call['Implied.Volatility']
)
BS_PUT_Pred = black_scholes_put_option(
    BS_Putt['futures.price'], 
    BS_Putt['Strike.Price'], 
    BS_Putt['Time.to.maturity'], 
    np.zeros(BS_Putt.shape[0]), 
    BS_Putt['Risk.Free.Rate'], 
    BS_Putt['Implied.Volatility']
)

print('The total variance explained by BS is {} for the Call Options'.format(
    np.round(r2_score(BS_Call['Settlement'], BS_Call_Pred), 5))
     )
print('The total variance explained by BS is {} for the Putt Options'.format(
    np.round(r2_score(BS_Putt['Settlement'], BS_PUT_Pred), 5))
     )

The total variance explained by BS is 0.9994 for the Call Options
The total variance explained by BS is 0.999 for the Putt Options


In [11]:
# linear homogeneity in BS
BS_Call['Strike.Price'] = (BS_Call['Strike.Price'].values * 100) / BS_Call['futures.price']
BS_Call['Settlement'] = (BS_Call['Settlement'].values * 100) / BS_Call['futures.price']

BS_Putt['Strike.Price'] = (BS_Putt['Strike.Price'].values * 100) / BS_Putt['futures.price']
BS_Putt['Settlement'] = (BS_Putt['Settlement'].values * 100) / BS_Putt['futures.price']

In [12]:
Call_ITM_idx = BS_Call['Strike.Price'] < 95.2
Call_ATM_idx = np.logical_and(BS_Call['Strike.Price'] < 103, BS_Call['Strike.Price'] >= 95.2)
Call_OTM_idx = BS_Call['Strike.Price'] > 103
print('The number of ITM Call options is', np.sum(Call_ITM_idx))
print('The number of ATM Call options is', np.sum(Call_ATM_idx))
print('The number of OTM Call options is', np.sum(Call_OTM_idx))

The number of ITM Call options is 6678
The number of ATM Call options is 2223
The number of OTM Call options is 4807


In [13]:
Put_ITM_idx = BS_Putt['Strike.Price'] > 103
Put_ATM_idx = np.logical_and(BS_Putt['Strike.Price'] < 103, BS_Putt['Strike.Price'] >= 95.2)
Put_OTM_idx = BS_Putt['Strike.Price'] < 95.2
print('The number of ITM Call options is', np.sum(Put_ITM_idx))
print('The number of ATM Call options is', np.sum(Put_ATM_idx))
print('The number of OTM Call options is', np.sum(Put_OTM_idx))

The number of ITM Call options is 3123
The number of ATM Call options is 2234
The number of OTM Call options is 10906


In [14]:
BS_Call.describe()

,Strike.Price,Settlement,Implied.Volatility,futures.price,Time.to.maturity,Risk.Free.Rate
count,13708.000000,13708.000000,13708.000000,13708.000000,13708.000000,13708.000000
mean,95.902629,12.265374,0.326937,3285.233528,0.301698,0.002869
std,21.455349,13.411424,0.178820,416.592958,0.245081,0.004624
min,3.265946,0.000000,0.085464,2381.500000,0.008219,0.000200
25%,81.948522,1.094095,0.202784,3053.300000,0.095890,0.000900
50%,95.809744,7.976937,0.283317,3294.100000,0.257534,0.001100
75%,108.173851,19.657789,0.388412,3623.000000,0.427397,0.001600
max,184.757506,97.099840,1.781009,3967.600000,0.964384,0.016000


In [15]:
BS_Putt.describe()

,Strike.Price,Settlement,Implied.Volatility,futures.price,Time.to.maturity,Risk.Free.Rate
count,16263.000000,16263.000000,16263.000000,16263.000000,16263.000000,16263.000000
mean,81.992030,4.140431,0.411271,3291.804101,0.313604,0.002916
std,25.186654,6.915304,0.237628,409.744382,0.250395,0.004684
min,2.520415,0.000000,0.086024,2381.500000,0.008219,0.000200
25%,67.817722,0.159583,0.241178,3053.300000,0.095890,0.000900
50%,85.329833,1.269646,0.348741,3294.300000,0.260274,0.001100
75%,99.641072,5.094648,0.506372,3623.000000,0.452055,0.001600
max,157.984451,58.017711,2.168625,3967.600000,0.964384,0.016000


In [16]:
BS_Call_X = BS_Call[['Strike.Price', 'Time.to.maturity', 'Risk.Free.Rate', 'Implied.Volatility']]
BS_Call_y = BS_Call['Settlement']
# Standardize the features
scaler_Call_X = MinMaxScaler().fit(BS_Call_X)
X_Call = pd.DataFrame(scaler_Call_X.transform(BS_Call_X), columns = BS_Call_X.columns.values) 

scaler_Call_y = MinMaxScaler().fit(BS_Call_y.values.reshape(-1, 1))
y_Call = scaler_Call_y.transform(BS_Call_y.values.reshape(-1, 1))

### Results of ANN on the simulated data

In [17]:
path = "/Users/gaojinglun/Desktop/RSG/ES_EZ_ANN_call_withLR0.001"
ANN_call = keras.models.load_model(path)

In [18]:
ANN_call_Sim = ANN_call.predict(X_Call)
print('The total variance explained by ANN is {} for the Call Options'.format(
    np.round(r2_score(y_Call, ANN_call.predict(X_Call)), 5)
))

The total variance explained by ANN is 0.98485 for the Call Options


In [19]:
BS_Put_X = BS_Putt[['Strike.Price', 'Time.to.maturity', 'Risk.Free.Rate', 'Implied.Volatility']]
BS_Put_y = BS_Putt['Settlement']
# Standardize the features
scaler_Put_X = MinMaxScaler().fit(BS_Put_X)
X_Put = pd.DataFrame(scaler_Put_X.transform(BS_Put_X), columns = BS_Put_X.columns.values) 

scaler_Put_y = MinMaxScaler().fit(BS_Put_y.values.reshape(-1, 1))
y_Put = scaler_Put_y.transform(BS_Put_y.values.reshape(-1, 1))

In [20]:
path2 = "/Users/gaojinglun/Desktop/RSG/ES_EZ_ANN_Put_withLR0.0001"
ANN_put = keras.models.load_model(path2)

In [21]:
ANN_put_Sim = ANN_put.predict(X_Put)
print('The total variance explained by ANN is {} for the Put Options'.format(
      np.round(r2_score(y_Put, ANN_put.predict(X_Put)), 5)))

The total variance explained by ANN is 0.90162 for the Put Options


**There are more 0 in the Put Option's simulation data.**

### Results of ANN trained on the ES & EZ data

In [22]:
ES_EZ_Call = ES_EZ_call[['futures.price', 'Strike.Price', 'Time.to.maturity', 
                           'Risk.Free.Rate', 'Implied.Volatility']]
scaler_Call_X = MinMaxScaler().fit(ES_EZ_Call)
X_Call_Trianed = pd.DataFrame(scaler_Call_X.transform(ES_EZ_Call), columns = ES_EZ_Call.columns.values) 

ES_EZ_Put = ES_EZ_put[['futures.price', 'Strike.Price', 'Time.to.maturity', 
                           'Risk.Free.Rate', 'Implied.Volatility']]
scaler_Put_X = MinMaxScaler().fit(ES_EZ_Put)
X_Put_Trained = pd.DataFrame(scaler_Put_X.transform(ES_EZ_Put), columns = ES_EZ_Put.columns.values) 

In [23]:
path3 = "/Users/gaojinglun/Desktop/RSG/ANN_call_TrainOnESEZ_withLR0.01"
ANN_Call_Trained = keras.models.load_model(path3)

path4 = "/Users/gaojinglun/Desktop/RSG/ANN_put_TrainOnESEZ_withLR0.001"
ANN_Put_Trained = keras.models.load_model(path4)

ANN_call_Trained = ANN_Call_Trained.predict(X_Call_Trianed)
ANN_put_Trained = ANN_Put_Trained.predict(X_Put_Trained)

In [24]:
ES_EZ_Call_ITM = np.round(r2_score(y_Call[Call_ITM_idx], ANN_call_Trained[Call_ITM_idx]), 5)
ES_EZ_Call_ATM = np.round(r2_score(y_Call[Call_ATM_idx], ANN_call_Trained[Call_ATM_idx]), 5)
ES_EZ_Call_OTM = np.round(r2_score(y_Call[Call_OTM_idx], ANN_call_Trained[Call_OTM_idx]), 5)
ES_EZ_Call_ALL = np.round(r2_score(y_Call, ANN_call_Trained), 5)

ES_EZ_Put_ITM = np.round(r2_score(y_Put[Put_ITM_idx], ANN_put_Trained[Put_ITM_idx]), 5)
ES_EZ_Put_ATM = np.round(r2_score(y_Put[Put_ATM_idx], ANN_put_Trained[Put_ATM_idx]), 5)
ES_EZ_Put_OTM = np.round(r2_score(y_Put[Put_OTM_idx], ANN_put_Trained[Put_OTM_idx]), 5)
ES_EZ_Put_ALL = np.round(r2_score(y_Put, ANN_put_Trained), 5)

In [25]:
BS_Call_ITM = np.round(r2_score(ES_EZ_call['Settlement'][Call_ITM_idx], BS_Call_Pred[Call_ITM_idx]), 5)
BS_Call_ATM = np.round(r2_score(ES_EZ_call['Settlement'][Call_ATM_idx], BS_Call_Pred[Call_ATM_idx]), 5)
BS_Call_OTM = np.round(r2_score(ES_EZ_call['Settlement'][Call_OTM_idx], BS_Call_Pred[Call_OTM_idx]), 5)
BS_Call_ALL = np.round(r2_score(ES_EZ_call['Settlement'], BS_Call_Pred), 5)

BS_Put_ITM = np.round(r2_score(ES_EZ_put['Settlement'][Put_ITM_idx], BS_PUT_Pred[Put_ITM_idx]), 5)
BS_Put_ATM = np.round(r2_score(ES_EZ_put['Settlement'][Put_ATM_idx], BS_PUT_Pred[Put_ATM_idx]), 5)
BS_Put_OTM = np.round(r2_score(ES_EZ_put['Settlement'][Put_OTM_idx], BS_PUT_Pred[Put_OTM_idx]), 5)
BS_Put_ALL = np.round(r2_score(ES_EZ_put['Settlement'], BS_PUT_Pred), 5)

In [26]:
ANN_Call_Sim_ITM = np.round(r2_score(y_Call[Call_ITM_idx], ANN_call_Sim[Call_ITM_idx]), 5)
ANN_Call_Sim_ATM = np.round(r2_score(y_Call[Call_ATM_idx], ANN_call_Sim[Call_ATM_idx]), 5)
ANN_Call_Sim_OTM = np.round(r2_score(y_Call[Call_OTM_idx], ANN_call_Sim[Call_OTM_idx]), 5)
ANN_Call_Sim_ALL = np.round(r2_score(y_Call, ANN_call_Sim), 5)

ANN_Put_Sim_ITM = np.round(r2_score(y_Put[Put_ITM_idx], ANN_put_Sim[Put_ITM_idx]), 5)
ANN_Put_Sim_ATM = np.round(r2_score(y_Put[Put_ATM_idx], ANN_put_Sim[Put_ATM_idx]), 5)
ANN_Put_Sim_OTM = np.round(r2_score(y_Put[Put_OTM_idx], ANN_put_Sim[Put_OTM_idx]), 5)
ANN_Put_Sim_ALL = np.round(r2_score(y_Put, ANN_put_Sim), 5)

In [27]:
table = pd.DataFrame({
    'ITM Call': [BS_Call_ITM, ANN_Call_Sim_ITM, ES_EZ_Call_ITM],
    'ATM Call': [BS_Call_ATM, ANN_Call_Sim_ATM, ES_EZ_Call_ATM],
    'OTM Call': [BS_Call_OTM, ANN_Call_Sim_OTM, ES_EZ_Call_OTM],
    'Overall Call': [BS_Call_ALL, ANN_Call_Sim_ALL, ES_EZ_Call_ALL],
    'ITM Put': [BS_Put_ITM, ANN_Put_Sim_ITM, ES_EZ_Put_ITM],
    'ATM Put': [BS_Put_ATM, ANN_Put_Sim_ATM, ES_EZ_Put_ATM],
    'OTM Put': [BS_Put_OTM, ANN_Put_Sim_OTM, ES_EZ_Put_OTM],
    'Overall Put': [BS_Put_ALL, ANN_Put_Sim_ALL, ES_EZ_Put_ALL]
},
    index = ['BS', 'ANN on Sim', 'ANN on ES_EZ']
)

In [28]:
print(tabulate(table, headers = 'keys', tablefmt = 'plain'))

                ITM Call    ATM Call    OTM Call    Overall Call    ITM Put    ATM Put    OTM Put    Overall Put
BS               0.99881     0.98681     0.99612         0.9994     0.99748    0.99219    0.99795        0.999
ANN on Sim       0.97802     0.66926     0.38616         0.98485    0.84855   -0.3341     0.54857        0.90162
ANN on ES_EZ     0.91641     0.66189     0.23069         0.9584     0.89738    0.8723     0.86952        0.95597


## Summary

- It seems like people highly rely on the BS model to price the [American(ES) options](https://www.cmegroup.com/trading/equity-index/weekly-eom-options-faq.html)

- It's harder to use the real-world data to price the options
  - less data
  - worse results
  - more hidden layers needed $\rightarrow$ more complex
  
- Training on the simulated data is useful, however, we need more data. We have shown before ANN can perform well for the ATM and OTM options if we have enough data. $\rightarrow$ Might need supercomputer

- Test the BS and ANN in the simulated market.